---

Created for [Pricing and Hedging Derivative Securities: Theory and Methods](https://book.derivative-securities.org/)

Authored by
- Kerry Back, Rice University
- Hong Liu, Washington University in St. Louis
- Mark Loewenstein, University of Maryland
 
---

<a target="_blank" href="https://colab.research.google.com/github/math-finance-book/book-code/blob/main/11_Exotics.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

In [ ]:

import plotly
from IPython.display import display, HTML

plotly.offline.init_notebook_mode(connected=True)
display(
    HTML(
        '<script type="text/javascript" async src="https://cdnjs.cloudflare.com/ajax/libs/mathjax/2.7.1/MathJax.js?config=TeX-MML-AM_SVG"></script>'
    )
)

In [ ]:

"""

def down_and_out_call(S, K, r, sigma, q, T, Barrier):
    if K > Barrier:
        a = S / K
        b = Barrier * Barrier / (K * S)
    else:
        a = S / Barrier
        b = Barrier / S

    d1 = (np.log(a) + (r - q + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    d1prime = (np.log(b) + (r - q + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    d2prime = d1prime - sigma * np.sqrt(T)
    N1 = norm.cdf(d1)
    N2 = norm.cdf(d2)
    N1prime = norm.cdf(d1prime)
    N2prime = norm.cdf(d2prime)
    x = 1 + 2 * (r - q) / (sigma ** 2)
    y = x - 2
    q1 = N1 - (Barrier / S) ** x * N1prime
    q2 = N2 - (Barrier / S) ** y * N2prime

    return np.exp(-q * T) * S * q1 - np.exp(-r * T) * K * q2
print("Down and Out Call:", down_and_out_call(S, K, r, sigma, q, T, 80))
"""

In [ ]:

def floating_strike_call(S, r, sigma, q, T, SMin):
    d1 = (np.log(S / SMin) + (r - q + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    d2prime = (np.log(SMin / S) + (r - q - 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    N1 = norm.cdf(d1)
    N2 = norm.cdf(d2)
    N2prime = norm.cdf(d2prime)
    x = 2 * (r - q) / (sigma ** 2)
    return np.exp(-q * T) * S * N1 - np.exp(-r * T) * SMin * N2 + (1 / x) * (SMin / S) ** x * np.exp(-r * T) * SMin * N2prime - (1 / x) * np.exp(-q * T) * S * (1 - N1)

# Example usage

S = 100
r = 0.05
sigma = 0.2
q = 0.02
T=1


# print("Floating Strike Call:", floating_strike_call(S, r, sigma, q, T, 90))

In [ ]:

import numpy as np

def black_scholes_call(S, K, r, sigma, q, T):
    """
    Inputs:
    S = initial stock price
    K = strike price
    r = risk-free rate
    sigma = volatility
    q = dividend yield
    T = time to maturity
    """
    if sigma == 0:
        return max(0, np.exp(-q * T) * S - np.exp(-r * T) * K)
    else:
        d1 = (np.log(S / K) + (r - q + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
        d2 = d1 - sigma * np.sqrt(T)
        N1 = norm.cdf(d1)
        N2 = norm.cdf(d2)
        return np.exp(-q * T) * S * N1 - np.exp(-r * T) * K * N2


def call_on_call(S, Kc, Ku, r, sigma, q, Tc, Tu):
    tol = 1e-6
    lower = 0
    upper = np.exp(q * (Tu - Tc)) * (Kc + Ku)
    guess = 0.5 * lower + 0.5 * upper
    flower = -Kc
    fupper = black_scholes_call(upper, Ku, r, sigma, q, Tu - Tc) - Kc
    fguess = black_scholes_call(guess, Ku, r, sigma, q, Tu - Tc) - Kc
    while upper - lower > tol:
        if fupper * fguess < 0:
            lower = guess
            flower = fguess
            guess = 0.5 * lower + 0.5 * upper
            fguess = black_scholes_call(guess, Ku, r, sigma, q, Tu - Tc) - Kc
        else:
            upper = guess
            fupper = fguess
            guess = 0.5 * lower + 0.5 * upper
            fguess = black_scholes_call(guess, Ku, r, sigma, q, Tu - Tc) - Kc
    Sstar = guess

    d1 = (np.log(S / Sstar) + (r - q + sigma ** 2 / 2) * Tc) / (sigma * np.sqrt(Tc))
    d2 = d1 - sigma * np.sqrt(Tc)
    d1prime = (np.log(S / Ku) + (r - q + sigma ** 2 / 2) * Tu) / (sigma * np.sqrt(Tu))
    d2prime = d1prime - sigma * np.sqrt(Tu)
    rho = np.sqrt(Tc / Tu)
    N2 = norm.cdf(d2)
    M1 = binormal_prob(d1, d1prime, rho)
    M2 = binormal_prob(d2, d2prime, rho)

    return -np.exp(-r * Tc) * Kc * N2 + np.exp(-q * Tu) * S * M1 - np.exp(-r * Tu) * Ku * M2

# Example usage
S = 100
Kc = 10
Ku = 100
r = 0.05
sigma = 0.2
q = 0.02
Tc = 0.5
Tu = 1


# print("Call on Call:", call_on_call(S, Kc, Ku, r, sigma, q, Tc, Tu))

In [ ]:

def black_scholes_put(S, K, r, sigma, q, T):
    """
    Inputs:
    S = initial stock price
    K = strike price
    r = risk-free rate
    sigma = volatility
    q = dividend yield
    T = time to maturity
    """
    if sigma == 0:
        return max(0, np.exp(-r * T) * K - np.exp(-q * T) * S)
    else:
        d1 = (np.log(S / K) + (r - q + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
        d2 = d1 - sigma * np.sqrt(T)
        N1 = norm.cdf(-d1)
        N2 = norm.cdf(-d2)
        return np.exp(-r * T) * K * N2 - np.exp(-q * T) * S * N1

def call_on_put(S, Kc, Ku, r, sigma, q, Tc, Tu):
    tol = 1e-6
    lower = 0
    flower = np.exp(-r * (Tu - Tc)) * Ku - Kc
    upper = 2 * Ku
    fupper = black_scholes_put(upper, Ku, r, sigma, q, Tu - Tc) - Kc
    while fupper > 0:
        upper *= 2
        fupper = black_scholes_put(upper, Ku, r, sigma, q, Tu - Tc) - Kc

    guess = 0.5 * lower + 0.5 * upper
    fguess = black_scholes_put(guess, Ku, r, sigma, q, Tu - Tc) - Kc
    while upper - lower > tol:
        if fupper * fguess < 0:
            lower = guess
            flower = fguess
            guess = 0.5 * lower + 0.5 * upper
            fguess = black_scholes_put(guess, Ku, r, sigma, q, Tu - Tc) - Kc
        else:
            upper = guess
            fupper = fguess
            guess = 0.5 * lower + 0.5 * upper
            fguess = black_scholes_put(guess, Ku, r, sigma, q, Tu - Tc) - Kc
    Sstar = guess

    d1 = (np.log(S / Sstar) + (r - q + sigma ** 2 / 2) * Tc) / (sigma * np.sqrt(Tc))
    d2 = d1 - sigma * np.sqrt(Tc)
    d1prime = (np.log(S / Ku) + (r - q + sigma ** 2 / 2) * Tu) / (sigma * np.sqrt(Tu))
    d2prime = d1prime - sigma * np.sqrt(Tu)
    rho = np.sqrt(Tc / Tu)
    N2 = norm.cdf(-d2)
    M1 = binormal_prob(-d1, -d1prime, rho)
    M2 = binormal_prob(-d2, -d2prime, rho)

    return -np.exp(-r * Tc) * Kc * N2 + np.exp(-r * Tu) * Ku * M2 - np.exp(-q * Tu) * S * M1
# Example usage
S = 100
Kc = 10
Ku = 100
r = 0.05
sigma = 0.2
q = 0.02
Tc = 0.5
Tu = 1

# print("Call on Put:", call_on_put(S, Kc, Ku, r, sigma, q, Tc, Tu))

In [ ]:

def call_on_max(S1, S2, K, r, sig1, sig2, rho, q1, q2, T):
    sigma = np.sqrt(sig2 ** 2 - 2 * rho * sig1 * sig2 + sig1 ** 2)
    d1 = (np.log(S1 / S2) + (q2 - q1 + sigma ** 2 / 2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    d11 = (np.log(S1 / K) + (r - q1 + sig1 ** 2 / 2) * T) / (sig1 * np.sqrt(T))
    d12 = d11 - sig1 * np.sqrt(T)
    d21 = (np.log(S2 / K) + (r - q2 + sig2 ** 2 / 2) * T) / (sig2 * np.sqrt(T))
    d22 = d21 - sig2 * np.sqrt(T)
    rho1 = (sig1 - rho * sig2) / sigma
    rho2 = (sig2 - rho * sig1) / sigma
    M1 = binormal_prob(d11, d1, rho1)
    M2 = binormal_prob(d21, -d2, rho2)
    M3 = binormal_prob(-d12, -d22, rho)

    return np.exp(-q1 * T) * S1 * M1 + np.exp(-q2 * T) * S2 * M2 + np.exp(-r * T) * K * M3 - np.exp(-r * T) * K
# Example usage

S1 = 100
S2= 100
K = 100
r = 0.05
sigma1 = 0.2
sigma2 = 0.2
q1 = 0.02
q2= 0.01
T=1
rho= 0.1

# print("Call on Max:", call_on_max(S1, S2, K, r, sigma1, sigma2, rho, q1, q2, T))

In [ ]:

import numpy as np
from scipy.stats import norm

def binormal_prob(a, b, rho):
    x = np.array([0.24840615, 0.39233107, 0.21141819, 0.03324666, 0.00082485334])
    y = np.array([0.10024215, 0.48281397, 1.0609498, 1.7797294, 2.6697604])
    a1 = a / np.sqrt(2 * (1 - rho ** 2))
    b1 = b / np.sqrt(2 * (1 - rho ** 2))
    if a <= 0 and b <= 0 and rho <= 0:
        total_sum = 0
        for i in range(5):
            for j in range(5):
                z1 = a1 * (2 * y[i] - a1)
                Z2 = b1 * (2 * y[j] - b1)
                z3 = 2 * rho * (y[i] - a1) * (y[j] - b1)
                total_sum += x[i] * x[j] * np.exp(z1 + Z2 + z3)
        return total_sum * np.sqrt(1 - rho ** 2) / np.pi
    elif a <= 0 and b >= 0 and rho >= 0:
        return norm.cdf(a) - binormal_prob(a, -b, -rho)
    elif a >= 0 and b <= 0 and rho >= 0:
        return norm.cdf(b) - binormal_prob(-a, b, -rho)
    elif a >= 0 and b >= 0 and rho <= 0:
        total_sum = norm.cdf(a) + norm.cdf(b)
        return total_sum - 1 + binormal_prob(-a, -b, rho)
    elif a * b * rho > 0:
        rho1 = (rho * a - b) * np.sign(a) / np.sqrt(a ** 2 - 2 * rho * a * b + b ** 2)
        rho2 = (rho * b - a) * np.sign(b) / np.sqrt(a ** 2 - 2 * rho * a * b + b ** 2)
        Delta = (1 - np.sign(a) * np.sign(b)) / 4
        return binormal_prob(a, 0, rho1) + binormal_prob(b, 0, rho2) - Delta
# print("BiNormalProb:", binormal_prob(0.1, 0.2, 0.3))

In [ ]:

def generic_option(P1, P2, sigma, T):
    """
    Inputs:
    P1 = present value of asset to be received
    P2 = present value of asset to be delivered
    sigma = volatility
    T = time to maturity
    """
    x = (np.log(P1 / P2) + 0.5 * sigma ** 2 * T) / (sigma * np.sqrt(T))
    y = x - sigma * np.sqrt(T)
    N1 = norm.cdf(x)
    N2 = norm.cdf(y)
    return P1 * N1 - P2 * N2

def forward_start_call(S, r, sigma, q, Tset, TCall):
    P1 = np.exp(-q * TCall) * S
    P2 = np.exp(-q * Tset - r * (TCall - Tset)) * S
    return generic_option(P1, P2, sigma, TCall - Tset)

# Example usage
S = 100
K = 100
r = 0.05
sigma = 0.2
q = 0.02
T = 1
Div = 5
TDiv = 0.5
TCall = 1
N = 10

# print("Forward Start Call:", forward_start_call(S, r, sigma, q, 0.5, TCall))

In [ ]:

def chooser(S, Kc, Kp, r, sigma, q, T, Tc, Tp):
    tol = 1e-6
    lower = 0
    upper = np.exp(q * Tc) * (Kc + Kp)
    guess = 0.5 * Kc + 0.5 * Kp
    flower = -np.exp(-r * (Tp - T)) * Kp
    fupper = black_scholes_call(upper, Kc, r, sigma, q, Tc - T) - black_scholes_put(upper, Kp, r, sigma, q, Tp - T)
    fguess = black_scholes_call(guess, Kc, r, sigma, q, Tc - T) - black_scholes_put(guess, Kp, r, sigma, q, Tp - T)
    while upper - lower > tol:
        if fupper * fguess < 0:
            lower = guess
            flower = fguess
            guess = 0.5 * lower + 0.5 * upper
            fguess = black_scholes_call(guess, Kc, r, sigma, q, Tc - T) - black_scholes_put(guess, Kp, r, sigma, q, Tp - T)
        else:
            upper = guess
            fupper = fguess
            guess = 0.5 * lower + 0.5 * upper
            fguess = black_scholes_call(guess, Kc, r, sigma, q, Tc - T) - black_scholes_put(guess, Kp, r, sigma, q, Tp - T)
    Sstar = guess

    d1 = (np.log(S / Sstar) + (r - q + sigma ** 2 / 2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    d1c = (np.log(S / Kc) + (r - q + sigma ** 2 / 2) * Tc) / (sigma * np.sqrt(Tc))
    d2c = d1c - sigma * np.sqrt(Tc)
    d1p = (np.log(S / Kp) + (r - q + sigma ** 2 / 2) * Tp) / (sigma * np.sqrt(Tp))
    d2p = d1p - sigma * np.sqrt(Tp)
    rhoc = np.sqrt(T / Tc)
    rhop = np.sqrt(T / Tp)
    M1c = binormal_prob(d1, d1c, rhoc)
    M2c = binormal_prob(d2, d2c, rhoc)
    M1p = binormal_prob(-d1, -d1p, rhop)
    M2p = binormal_prob(-d2, -d2p, rhop)

    return np.exp(-q * Tc) * S * M1c - np.exp(-r * Tc) * Kc * M2c + np.exp(-r * Tp) * Kp * M2p - np.exp(-q * Tp) * S * M1p

# Example usage
S = 100
Kc = 80
Kp = 80
r = 0.05
sigma = 0.2
q = 0.02
Tc = 1.5
Tp = 1.5
T=1

# print("Chooser Option:", chooser(S, Kc, Kp, r, sigma, q, T, Tc, Tp))